# Estimate Offline Model Parameters

## Import Stuff

In [21]:
from mcs_selection_env import MCSSelectionEnv

import numpy as np

## Create Environment

In [22]:
def create_training_dataset(cqi, dataset_size = 100):
    # Create environment
    env = MCSSelectionEnv(ue_relative_speed = 20)

    # Create variables to collect data
    ack = []
    snr_per_subcarrier = []
    
    # Collect equal number of ACKs and NACKs for unbiased training
    nrof_samples = int(dataset_size / 2)
    
    seed_value = 0
    i = j = 0 # ACK and NACK counter
    while( (i < nrof_samples) or (j < nrof_samples) ):
        # Update seed and reset env
        env.seed(seed_value)
        env.reset()
        
        # Get a data sample
        _, _, _, s, _, snrs = env.step(cqi)
        
        if (s == True) and (i < nrof_samples):
            ack.append(s)
            snr_per_subcarrier.append(snrs)
            
            i += 1
        elif (s == False) and (j < nrof_samples):
            ack.append(s)
            snr_per_subcarrier.append(snrs)
            
            j += 1
            
        seed_value += 1
        
    return (np.array(ack, dtype=int), np.array(snr_per_subcarrier))

## Create Training Dataset

In [23]:
training_data = {}
for cqi in range(16):
    print("CQI: %d"%(cqi))
    
    ack_data, snr_data = create_training_dataset(cqi, 100)
    training_data[cqi] = {"ack": ack_data, "snr_per_subcarrier": snr_data}

CQI: 0
CQI: 1
CQI: 2
CQI: 3
CQI: 4
CQI: 5
CQI: 6
CQI: 7
CQI: 8
CQI: 9
CQI: 10
CQI: 11
CQI: 12
CQI: 13
CQI: 14
CQI: 15


## Load Offline Model

In [24]:
offline_model_datafile = "src/Offline_Link_Model.npy"
offline_model = np.load( offline_model_datafile, encoding='latin1' )[ ( ) ]

In [25]:
snr_vs_bler = offline_model["snr_vs_bler"]
snr_range_dB = offline_model["snr_range_dB"]

snr_range = 10 ** (snr_range_dB / 0.1)

## Estimate Optimal Beta Values

In [26]:
def eesm(snr_per_subc, beta): 
    return (-1.0 * beta) * np.log( np.sum( np.exp( -1.0 * snr_per_subc / beta ) ) / len( snr_per_subc ) )

In [29]:
learnt_beta = []
for cqi in range( 16 ):
    data = training_data[cqi]
    
    ack = data["ack"]
    snr_per_subc = data["snr_per_subcarrier"]
    
    nrof_samples, _ = snr_per_subc.shape

    cost = []
    beta_grid = np.arange(0.1, 10, 0.1)
    for beta in beta_grid:
        awgn_bler = []
        for j in range(nrof_samples):
            eff_snr = eesm(snr_per_subc[j,:], beta)

            high_index = np.argmin( snr_range_dB < eff_snr )
            bler_low = snr_vs_bler[ high_index, cqi ]
            bler_high = snr_vs_bler[ high_index - 1, cqi ]
            
            awgn_bler.append( ( ( bler_low + bler_high ) / 2.0 ) )

            #print('EESM: %0.2f, ACK: %d, AWGN BLER: %0.2f'%(eff_snr, ack[i], awgn_bler))
        
        #cross_entropy_loss = -1.0 * np.sum( ack * np.log( np.array( awgn_bler ) + 1e-9 ) 
        #                                    + ( ( 1 - ack ) * np.log( ( 1.0 - np.array( awgn_bler ) ) + 1e-9 ) ) ) / nrof_samples
        
        mse_loss = np.mean( ( ( 1 - ack ) - awgn_bler ) ** 2 )
        
        cost.append( mse_loss )
    
    #print(cost)
    opt_beta = beta_grid[ np.argmin( cost ) ]
    print("CQI: %d, Cost: %0.2f, Beta: %0.2f"%(cqi, np.min(cost), opt_beta))
    
    learnt_beta.append( opt_beta )

/home/ubuntu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


CQI: 0, Cost: 0.52, Beta: 0.10
CQI: 1, Cost: 0.40, Beta: 0.10
CQI: 2, Cost: 0.17, Beta: 0.20
CQI: 3, Cost: 0.10, Beta: 0.80
CQI: 4, Cost: 0.07, Beta: 1.40
CQI: 5, Cost: 0.07, Beta: 1.90
CQI: 6, Cost: 0.06, Beta: 2.20
CQI: 7, Cost: 0.09, Beta: 2.90
CQI: 8, Cost: 0.09, Beta: 3.40
CQI: 9, Cost: 0.11, Beta: 3.20
CQI: 10, Cost: 0.11, Beta: 3.10
CQI: 11, Cost: 0.13, Beta: 4.50
CQI: 12, Cost: 0.09, Beta: 3.90
CQI: 13, Cost: 0.11, Beta: 4.00
CQI: 14, Cost: 0.09, Beta: 3.50
CQI: 15, Cost: 0.10, Beta: 4.30


In [30]:
print(learnt_beta)

[0.1, 0.1, 0.2, 0.8, 1.4000000000000001, 1.9000000000000001, 2.2, 2.9000000000000004, 3.4000000000000004, 3.2, 3.1, 4.5, 3.9000000000000004, 4.0, 3.5000000000000004, 4.3]
